So far we've learnt how to scrape the web, and how to make a request for information from an API. Some websites make APIs even easier. Check out [RapidAPI](https://rapidapi.com/) they take care of writing most of the code for you.

We will use the [AeroDataBox API](https://rapidapi.com/aedbx-aedbx/api/aerodatabox/), which can retrieve all sorts of information about flights and airports. We will show you how to retrieve information about the airports, and then it's up to you to apply this, along with what you've already learnt this week, to **produce a function, which retrieves tomorrows flight information for the major airports in the cities you web scraped**.

In [1]:
import pandas as pd
import requests

On the left hand side of the AeroDataBox API page, you'll see a list of options for information that you can retrieve:
> - Flights API
- Subsciption / PUSH API
- Airport API
- Aircraft API
- Healthcheck & Status API

1. We want to select `Airport API`

2. Then within Airport API we want to select `Search airports by location`

3. Now in the middle third you'll want to enter the `latitude` and `longitude` of any city to test... we chose Berlin: latitude 52.31 longitude 13.24. Next we changed the `radiusKM` to only 50km. And finally set `withFlightInfoOnly` to true, so it will only return airports which have flight data (scheduled or live) available.

4. On the right hand third of the screen you should see a block of code that looks pretty unfamiliar. This is because by default the code is probably set to *(Node.js) Axios*. However, we have the power to change this to familiar python. Select the dropdown box at the top of the code and select `python > requests`.

Now you can copy the code to your notebook and it should look a little something like the cell below:

In [2]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/search/location/52.31/13.24/km/50/16"

querystring = {"withFlightInfoOnly":"true"}

headers = {
	"X-RapidAPI-Key": "YOUR_API_KEY_HERE",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"message":"You are not subscribed to this API."}


Let's view the response as `.json()` instead of `.text` so that it's easier to read

In [36]:
#response.json()

We can now turn this into a dataframe using `.json_normalize()`

In [99]:
#pd.json_normalize(response.json()['items'])

Let's now use this for the latitude and longitude of multiple cities

In [3]:
def icao_airport_codes(latitudes, longitudes):

  #assert len(latitudes) == len(longitudes)
  
  list_for_df = []

  for i in range(len(latitudes)):

    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitudes[i]}/{longitudes[i]}/km/100/16"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
      "X-RapidAPI-Key": "YOUR_API_KEY_HERE"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    list_for_df.append(pd.json_normalize(response.json()['items']))

  return pd.concat(list_for_df, ignore_index=True)

###### **Challenge 1:** Icao codes
If you use the above for all of your cities, you can create a DataFrame of all the airports and their associated `icao` codes. Perfect for a relational database.

###### **Challenge 2:** Arrivals information
Using what you have been shown above, plus the skills you've learnt in the last couple of days:
1. In `AeroDataBox API` use the `Flight API` > `FIDS/Schedules: Airport departures and arrivals by airport ICAO code` section
2. Fill out the parameters in the middle third and then copy the `python: requests` code from the right hand third
3. Explore the data you get back. What would be useful in your DataFrame and what can be excluded? Remember Gans wants to know about when people are arriving in the city
4. Make a DataFrame from the information you see as important
5. Condense everything you did above into a function that can take a list of ICAO codes as an input, and as an output gives you a DataFrame with the information for *tomorrows arrivals*

In [4]:
# your code here
import requests

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EHAM/2022-11-04T08:00/2022-11-04T19:59"

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
	"X-RapidAPI-Key": "e723d7487fmsh7fce19b65c2f2eep119c33jsneeee8c0c0fc9",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

#print(response.text)

In [4]:
import pandas as pd

In [6]:
flight_data = response.json()

In [7]:
type(flight_data)

dict

In [6]:
len(flight_data)

2

In [11]:
flight_data.keys()

dict_keys(['message'])

In [12]:
flight_data['message']

'You are not subscribed to this API.'

In [83]:
#flight_data['departures']

In [9]:
arrival_data = flight_data['arrivals']

KeyError: 'arrivals'

# Arrivals

In [ ]:
#flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)

In [9]:
if flight_data['arrivals'][10]['arrival'] 


SyntaxError: invalid syntax (2578028318.py, line 1)

In [10]:
len(arrival_data)

1435

In [11]:
type(arrival_data)

list

In [12]:
arrival_data[0]['arrival']['actualTimeLocal']

'2022-11-04 15:50+01:00'

In [13]:
schedulded_time = [] 
schedulded_time.append(arrival_data[0]['arrival']['scheduledTimeLocal'])
schedulded_time

['2022-11-03 08:45+01:00']

We need 
1. actual arrival time
2. schedulded arrival 
3. airline name
4. terminal


In [14]:
arrival_data[0].keys()

dict_keys(['departure', 'arrival', 'number', 'status', 'codeshareStatus', 'isCargo', 'aircraft', 'airline'])

In [17]:
arrival_data[0]

{'departure': {'airport': {'icao': 'LSZH', 'iata': 'ZRH', 'name': 'Zurich'},
  'quality': []},
 'arrival': {'scheduledTimeLocal': '2022-11-03 08:45+01:00',
  'actualTimeLocal': '2022-11-04 15:50+01:00',
  'runwayTimeLocal': '2022-11-04 15:50+01:00',
  'scheduledTimeUtc': '2022-11-03 07:45Z',
  'actualTimeUtc': '2022-11-04 14:50Z',
  'runwayTimeUtc': '2022-11-04 14:50Z',
  'terminal': '2',
  'baggageBelt': '11',
  'quality': ['Basic', 'Live']},
 'number': 'DL 9549',
 'status': 'Arrived',
 'codeshareStatus': 'IsCodeshared',
 'isCargo': False,
 'aircraft': {'model': 'Embraer 195-E2'},
 'airline': {'name': 'Delta Air Lines'}}

In [98]:
#flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)


## Terminal

 DID NOT FIND TERMINAL DATA FOR THIS CIAO/CAIO

In [18]:
arrival_data[100]['arrival']

{'scheduledTimeLocal': '2022-11-04 08:45+01:00',
 'actualTimeLocal': '2022-11-04 08:17+01:00',
 'runwayTimeLocal': '2022-11-04 08:17+01:00',
 'scheduledTimeUtc': '2022-11-04 07:45Z',
 'actualTimeUtc': '2022-11-04 07:17Z',
 'runwayTimeUtc': '2022-11-04 07:17Z',
 'terminal': '2',
 'baggageBelt': '9',
 'quality': ['Basic', 'Live']}

In [104]:
d1 = {}
d1['key or value'] = [2,9,9,0]
d1

{'key or value': [2, 9, 9, 0]}

In [123]:
terminal = []
#for i in range(len(arrival_data)):
#  if arrival_data[i]['arrival']['terminal'] is None:
 #   terminal.append("None")
 # else:
  #  terminal.append(arrival_data[i]['arrival']['terminal'])

#terminal

In [117]:
arrival_data[0]['departure'].keys()

dict_keys(['airport', 'scheduledTimeLocal', 'scheduledTimeUtc', 'terminal', 'quality'])

In [122]:
flight_data['departures'][0]['departure'].keys()

dict_keys(['scheduledTimeLocal', 'actualTimeLocal', 'scheduledTimeUtc', 'actualTimeUtc', 'quality'])

## Status

In [37]:
arrival_data[0]['status']

'Expected'

In [63]:
status = [] 
for s in range(len(arrival_data)):
  if arrival_data[s]['status'] is None:
    status.append("none")
  else:
    status.append(arrival_data[s]['status'])

flight_status = {'status':[]}
flight_status['status'] += status
flight_status.keys()

dict_keys(['status'])

In [61]:
status[-6:-1]

['Expected', 'Expected', 'Expected', 'Expected', 'Expected']

## Departure Airport

In [23]:
arrival_data[0]['departure']['airport']['name']

'Paris'

In [64]:
departure_city = [] 
for name in range(len(arrival_data)):
  if arrival_data[name]['departure']['airport']['name'] is None:
    departure_city.append("none")
  else:
    departure_city.append(arrival_data[name]['departure']['airport']['name'])

dep = {'departure_airport':[]}
dep['departure_airport'] += departure_city
dep.keys()

dict_keys(['departure_airport'])

In [66]:
#dep

In [26]:
#departure_city

## Flight Number

In [29]:
arrival_data[0]['number']

'AF 1240'

In [67]:
flight_number = [] 
for number in range(len(arrival_data)):
  if arrival_data[number]['number'] is None:
    flight_number.append("none")
  else:
    flight_number.append(arrival_data[number]['number'])
num = {'flight_number':[]}
num['flight_number'] += flight_number
num.keys()

dict_keys(['flight_number'])

In [31]:
flight_number[1:6]

['AF 3126', 'AF 3193', 'AF 5132', 'AF 5199', 'AF 5597']

In [69]:
#num

## Flights Name

In [73]:
arrival_data = flight_data['arrivals']
arrival_data[0]

{'departure': {'airport': {'icao': 'LFPG', 'iata': 'CDG', 'name': 'Paris'},
  'scheduledTimeLocal': '2022-11-04 07:10+01:00',
  'scheduledTimeUtc': '2022-11-04 06:10Z',
  'terminal': '2F',
  'quality': ['Basic']},
 'arrival': {'scheduledTimeLocal': '2022-11-04 08:35+01:00',
  'actualTimeLocal': '2022-11-04 08:27+01:00',
  'runwayTimeLocal': '2022-11-04 08:27+01:00',
  'scheduledTimeUtc': '2022-11-04 07:35Z',
  'actualTimeUtc': '2022-11-04 07:27Z',
  'runwayTimeUtc': '2022-11-04 07:27Z',
  'quality': ['Basic', 'Live']},
 'number': 'AF 1240',
 'status': 'Expected',
 'codeshareStatus': 'IsOperator',
 'isCargo': False,
 'aircraft': {'model': 'Airbus A321'},
 'airline': {'name': 'Air France'}}

In [77]:
arrival_data[0]['airline']

{'name': 'Air France'}

In [76]:
type(arrival_data[0]['airline'])

dict

In [86]:
airline = list(arrival_data[0]['airline'].values())[0]
airline

'Air France'

### List

In [70]:
airline_data = [] 
for name in range(len(arrival_data)):
  if list(arrival_data[name]['airline'].values())[0] is None:
    airline_data.append("none")
  else:
    airline_data.append(list(arrival_data[name]['airline'].values())[0])

airl = {'Airline':[]}
airl['Airline'] += airline_data
airl.keys()

#airline_data[5:12]

dict_keys(['Airline'])

In [72]:
#airl

## Actual Arrival Time

*   List item
*   List item



In [87]:
arrival_data[214]['arrival']

{'scheduledTimeLocal': '2022-11-04 08:05+01:00',
 'scheduledTimeUtc': '2022-11-04 07:05Z',
 'quality': ['Basic']}

In [66]:
actual_time = []
actual_time.append(arrival_data[216]['arrival']['actualTimeLocal'])
actual_time

['2022-11-04 08:27+01:00']

In [67]:
len(arrival_data)

1446

In [73]:
actual_time = [] 
#if id is None:
#           id_list.append("No Form Found")
#        else:
 #           id_list.append(id.get("id"))
for i in range(214):
  if arrival_data[i]['arrival']['actualTimeLocal'] is None:
    actual_time.append(0)
  else:
    actual_time.append(arrival_data[i]['arrival']['actualTimeLocal'])




## Schedulded Arrival Time

In [74]:
schedulded_time = [] 
for time in range(len(arrival_data)):
  schedulded_time.append(arrival_data[time]['arrival']['scheduledTimeLocal'])

schedulded_time[1:6]

time = {'Arrival_Time':[]}
time['Arrival_Time'] += schedulded_time
time.keys()

dict_keys(['Arrival_Time'])

In [76]:
#time

## Make A Dataframe

1. Schedulded_Arrival: schedulded_time 
2. arrival_terminal
3. Departure_City: departure_city
4. Airline_name: airline_data 
5. Flight_number: flight_number 
6. Status: status

        
departures_df = pd.DataFrame({
    'Type': De_Type,
    'Departure_Time' : De_Departure_Time,
    'Destination_Name' : De_Destination_Name,
    'Status' : De_Status,

})

In [131]:
flights = pd.DataFrame ({ 'schedulded_arrival':schedulded_time,
                      'departure_city':departure_city,
                      'airline_name': airline_data,
                      'flight_number': flight_number,
                      'status': status
                          })

In [133]:
flights

,schedulded_arrival,departure_city,airline_name,flight_number,status
0,2022-11-04 08:35+01:00,Paris,Air France,AF 1240,Expected
1,2022-11-04 08:15+01:00,Bangalore,Air France,AF 3126,Expected
2,2022-11-04 08:10+01:00,Norwich,Air France,AF 3193,Expected
3,2022-11-04 08:20+01:00,Leeds,Air France,AF 5132,Expected
4,2022-11-04 08:20+01:00,Bristol,Air France,AF 5199,Expected
...,...,...,...,...,...
1441,2022-11-04 20:00+01:00,Paris,Xiamen,MF 9754,Expected
1442,2022-11-04 20:05+01:00,Dublin,Xiamen,MF 9998,Expected
1443,2022-11-04 20:00+01:00,Gdańsk,KLM,KL 1922,Expected
1444,2022-11-04 20:05+01:00,Dublin,KLM,KL 938,Expected


## For loop for data collection of arrival


In [13]:
schedulded_time = [] 
for time in range(len(arrival_data)):
  schedulded_time.append(arrival_data[time]['arrival']['scheduledTimeLocal'])


airline_data = [] 
for name in range(len(arrival_data)):
  if list(arrival_data[name]['airline'].values())[0] is None:
    airline_data.append("none")
  else:
    airline_data.append(list(arrival_data[name]['airline'].values())[0])

departure_city = [] 
for name in range(len(arrival_data)):
  if arrival_data[name]['departure']['airport']['name'] is None:
    departure_city.append("none")
  else:
    departure_city.append(arrival_data[name]['departure']['airport']['name'])

status = [] 
for s in range(len(arrival_data)):
  if arrival_data[s]['status'] is None:
    status.append("none")
  else:
    status.append(arrival_data[s]['status'])

flight_number = [] 
for number in range(len(arrival_data)):
  if arrival_data[number]['number'] is None:
    flight_number.append("none")
  else:
    flight_number.append(arrival_data[number]['number'])


NameError: name 'arrival_data' is not defined

In [136]:
schedulded_time = [] 
departure_city = [] 
airline_data = [] 
flight_number = []
status = [] 

for i in range(len(arrival_data)):
  schedulded_time.append(arrival_data[i]['arrival']['scheduledTimeLocal'])
  departure_city.append(arrival_data[i]['departure']['airport']['name'])
  airline_data.append(list(arrival_data[i]['airline'].values())[0])
  flight_number.append(arrival_data[i]['number'])
  status.append(arrival_data[i]['status'])




In [147]:
arrival_data[0]['departure']['airport']['icao']

'EFHK'

In [153]:
icao_list = ['EGLL', 'EDDB']
for icao in icao_list:
  url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/2022-11-04T10:00/2022-11-04T19:59"
  querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}
  headers = {
  "X-RapidAPI-Key": "e723d7487fmsh7fce19b65c2f2eep119c33jsneeee8c0c0fc9",
  "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
  }
  response = requests.request("GET", url, headers=headers, params=querystring)
  #print(response.text) 

  flight_data = response.json()
  arrival_data = flight_data['arrivals']
  flight_data = response.json()
  arrival_data = flight_data['arrivals']
    
    
  schedulded_time = [] 
  departure_city = [] 
  airline_data = [] 
  flight_number = []
  status = [] 
  arrival_icao = []
  departure_icao = []
  for data in arrival_data:
    arrival_icao.append(icao)
    schedulded_time.append(data['arrival']['scheduledTimeLocal'])
    departure_city.append(data['departure']['airport']['name'])
    airline_data.append(list(data['airline'].values())[0])
    flight_number.append(data['number'])
    status.append(data['status'])

    try:
      departure_icao.append(data['departure']['airport']['icao'])
    except:
      departure_icao.append('none')

flights = pd.DataFrame ({'arrival_icao': arrival_icao,
                      'schedulded_arrival':schedulded_time,
                      'departure_city':departure_city,
                      'departure_icao': departure_icao,
                      'airline_name': airline_data,
                      'flight_number': flight_number,
                      'status': status
                          })

flights.head(1)

,arrival_icao,schedulded_arrival,departure_city,departure_icao,airline_name,flight_number,status
0,EDDB,2022-11-04 10:50+01:00,Athens,LGAV,Aegean,A3 820,Unknown


## Clubbing the Data 

In [3]:
import pandas as pd
from datetime import datetime, date, timedelta
import requests
from pytz import timezone


### MAIN FUNCTION

In [11]:
def tmw_flight_arrival(icao_list):
    arrivals_list = []
    icaos = ["EDDB", "EDDH", "EGLL", "EGCC", "LEBL"]
    
    today = datetime.now().date() # only needed on colab as the computer is sometimes in a different timezone
    tomorrow = (today + timedelta(days=1))
    
    morning = {"start_time": "00:00", "end_time": "12:00"}
    afternoon = {"start_time": "12:00", "end_time": "23:59"}
    
    times = [morning, afternoon]

    for icao in icao_list:
          for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time['start_time']}/{tomorrow}T{time['end_time']}"
            querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}
            headers = {
                  "X-RapidAPI-Key": "e723d7487fmsh7fce19b65c2f2eep119c33jsneeee8c0c0fc9",
                  "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                   }
            response = requests.request("GET", url, headers=headers, params=querystring)
            flight_data = response.json()
            arrival_data = flight_data['arrivals']
          #  flight_data = response.json()
           # arrival_data = flight_data['arrivals']


            schedulded_time = [] 
            departure_city = [] 
            airline_data = [] 
            flight_number = []
            status = [] 
            arrival_icao = []
            departure_icao = []
            for data in arrival_data:
                arrival_icao.append(icao)
                schedulded_time.append(data['arrival']['scheduledTimeLocal'])
                departure_city.append(data['departure']['airport']['name'])
                airline_data.append(list(data['airline'].values())[0])
                flight_number.append(data['number'])
                status.append(data['status'])
                try:
                    departure_icao.append(data['departure']['airport']['icao'])
                except:
                    departure_icao.append('none')

                flights = pd.DataFrame({'arrival_icao': arrival_icao,
                          'schedulded_arrival':schedulded_time,
                          'departure_city':departure_city,
                          'departure_icao': departure_icao,
                          'airline_name': airline_data,
                          'flight_number': flight_number,
                          'status': status
                              })

            # CHANGING TIME TO DATETIME FORMAT
                flights["schedulded_arrival"] = pd.to_datetime(flights["schedulded_arrival"])
            arrivals_list.append(flights)


    
    return pd.concat(arrivals_list)

flight_df = tmw_flight_arrival(icaos)


In [12]:
flight_df

,arrival_icao,schedulded_arrival,departure_city,departure_icao,airline_name,flight_number,status
0,EDDB,2022-11-09 06:50:00+01:00,Doha,OTHH,Qatar Airways,QR 79,Unknown
1,EDDB,2022-11-09 07:55:00+01:00,Riga,EVRA,Air Baltic,BT 211,Unknown
2,EDDB,2022-11-09 07:50:00+01:00,Stuttgart,EDDS,Eurowings,EW 2002,Unknown
3,EDDB,2022-11-09 07:35:00+01:00,Duesseldorf,EDDL,Eurowings,EW 9048,Unknown
4,EDDB,2022-11-09 07:55:00+01:00,Frankfurt-am-Main,EDDF,Lufthansa,LH 170,Unknown
...,...,...,...,...,...,...,...
492,LEBL,2022-11-09 23:55:00+01:00,Lisbon,LPPT,Vueling,VY 8463,Unknown
493,LEBL,2022-11-09 23:50:00+01:00,Brussels,EBBR,Vueling,VY 8985,Canceled
494,LEBL,2022-11-09 23:45:00+01:00,Palma De Mallorca,LEPA,Wasaya Airways,WT 223,Unknown
495,LEBL,2022-11-09 22:30:00+01:00,Madrid,LEMD,Vueling,VY 5091,Expected


In [73]:
flight_df["flightid"] = flight_df.index

In [74]:

flight_df  = flight_df.reindex(columns = ["flightid","arrival_icao","schedulded_arrival","departure_city","airline_name","flight_number","status"])

In [77]:
flight = flight_df
flight

,flightid,arrival_icao,schedulded_arrival,departure_city,airline_name,flight_number,status
0,0,EDDB,2022-11-08 07:30:00+01:00,Cologne,Eurowings,EW 12,Unknown
1,1,EDDB,2022-11-08 07:50:00+01:00,Stuttgart,Eurowings,EW 2002,Unknown
2,2,EDDB,2022-11-08 07:35:00+01:00,Duesseldorf,Eurowings,EW 9048,Unknown
3,3,EDDB,2022-11-08 07:55:00+01:00,Frankfurt-am-Main,Lufthansa,LH 170,Unknown
4,4,EDDB,2022-11-08 06:50:00+01:00,Doha,Qatar Airways,QR 79,Unknown
...,...,...,...,...,...,...,...
475,475,LEBL,2022-11-08 23:45:00+01:00,Palma De Mallorca,Wasaya Airways,WT 223,Unknown
476,476,LEBL,2022-11-08 22:30:00+01:00,Madrid,Vueling,VY 5091,Expected
477,477,LEBL,2022-11-08 22:45:00+01:00,Milan,Vueling,VY 6337,Expected
478,478,LEBL,2022-11-08 22:20:00+01:00,London,Vueling,VY 7825,Expected


In [78]:
schema="e_scooter"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password="saideshmukh16" # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

print("b")

b


In [79]:
flight.to_sql("flights", if_exists = "append", con = con, index = False)

3184

# By Normalisation


In [72]:
import pandas as pd
from datetime import datetime, date, timedelta
import requests
from pytz import timezone

In [107]:
import requests

In [54]:
def test_test(icao):

  today = datetime.now().astimezone(timezone('Europe/Berlin')).date() # only needed on colab as the computer is sometimes in a different timezone
  tomorrow = (today + timedelta(days=1))
  time = ["00:00","11:59"]
  
  url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EHAM/2022-11-04T08:00/2022-11-04T19:59"
  querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}
  headers = {
	    "X-RapidAPI-Key": "e723d7487fmsh7fce19b65c2f2eep119c33jsneeee8c0c0fc9",
	    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
      }

  response = requests.request("GET", url, headers=headers, params=querystring)
  flights_json = response.json()

  df_ = pd.json_normalize(flights_json["arrivals"])[['number', 'status', 'departure.airport.iata', ]].rename(columns={'departure.airport.iata': 'departure_airport', 'scheduledTimeLocal':'Time'})

  return df_

In [55]:
flight_data['arrivals'][0]['arrival']

{'scheduledTimeLocal': '2022-11-04 08:35+01:00',
 'actualTimeLocal': '2022-11-04 08:27+01:00',
 'runwayTimeLocal': '2022-11-04 08:27+01:00',
 'scheduledTimeUtc': '2022-11-04 07:35Z',
 'actualTimeUtc': '2022-11-04 07:27Z',
 'runwayTimeUtc': '2022-11-04 07:27Z',
 'quality': ['Basic', 'Live']}

In [108]:
test_test("EHAM")

,number,status,departure_airport
0,AM 6471,Delayed,ZRH
1,DL 9549,Delayed,ZRH
2,G3 5579,Delayed,ZRH
3,KL 1952,Delayed,ZRH
4,KQ 1952,Delayed,ZRH
...,...,...,...
1449,KL 1922,Expected,GDN
1450,KL 938,Expected,DUB
1451,MF 9754,Expected,CDG
1452,MF 9998,Expected,DUB
